<a href="https://colab.research.google.com/github/GuilhermeDuar/ProjetoImersao/blob/main/ProjetoImersao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Baixando as dependências generativeai

In [47]:
!pip install -q -U google-generativeai

Configuração e criação do modelo generativo

In [102]:
import hashlib
import numpy as np
import pandas as pd
import google.generativeai as genai
import json
from types import SimpleNamespace
from google.colab import userdata
from pathlib import Path

key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=key)

# Config mais conservadora para evitar delírios.
generation_config = {
  "temperature": 0,
  "top_p": 0.95,
  "top_k": 0,
}

# Optei por remover as configuações de segurança por se tratar de um tema que envolve palavras "fortes".
# Desta forma o modelo não terá problemas em transcrever os áudios.
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

chat = model.start_chat(history=[
    {
      "role": "user",
      "parts": [genai.upload_file("/content/Audios/Acidente.mp3")]
    }
  ])

Base de Dados representada por uma lista de dicinários. \
Em um cenário real, poderia ser substituído por uma consulta em tabelas que representem essa relação de Ferimentos e Exames

In [103]:
ferimentosExames = [{
    "Ferimento": "Traumatismo craniano",
    "Exame": "Tomografia computadorizada (TC) da cabeça"},
     {
    "Ferimento": "Fratura óssea",
    "Exame": "Raio-X"},
     {
    "Ferimento": "Lesão na coluna vertebral",
    "Exame": "Ressonância Magnética (RM) da coluna"},
     {
    "Ferimento": "Lesão interna",
    "Exame": "Ultrassonografia (FAST), TC do abdômen"},
     {
    "Ferimento": "Queimaduras",
    "Exame": "Avaliação clínica, biópsia"},
     {
    "Ferimento": "Feridas abertas",
    "Exame": "Avaliação clínica"}]

df = pd.DataFrame(ferimentosExames)
df.columns = ["Ferimentos", "Exames"]

Criação de uma terceira coluna para realizar as comparações por embedding

In [104]:
def embed(title, text):
  return genai.embed_content(model="models/embedding-001",content=text, title=title, task_type="RETRIEVAL_DOCUMENT")["embedding"]

df["Embedding"] = df.apply(lambda row: embed(row["Ferimentos"], row["Exames"]), axis = 1)

Transcrição do contexto(áudio) e geração do Json

In [105]:
def gerarOcorrencia():
  response = chat.send_message("""Transcreva o audio em formato JSON, as chaves 'ResumoSituação', 'Ferimentos', 'TratamentosRealizados' e 'Exames'.
                                            Resumo da situação deve ser breve. Traumas sofridos e Tratamentos realizados recebem um vetor.
                                            Exames deve ser deixado em branco. Sem adicionar informações que não contenha no audio.""")

  # Replace caso o modelo insira quebra de linha, espaços ou informações desnecessárias na response.text
  return json.loads(response.text.replace("  ", "").replace("\n", "").replace("json", "").replace("`", ""))

Função para identificar o melhor exame com base em um ferimento

In [106]:
def consultaExame(ferimento):
  embeddingFerimento = genai.embed_content(model="models/embedding-001",content=ferimento, task_type="RETRIEVAL_QUERY")

  escalar = np.dot(np.stack(df["Embedding"]), embeddingFerimento["embedding"])

  return df.iloc[np.argmax(escalar)]["Exames"]

Função para gerar a lista de exames\
Para cada ferimento será gerado um exame adequado, não será repetido exames.

In [107]:
def geraListaExames(listaFerimentos):
  listaExames = []
  for ferimento in listaFerimentos:
    exame = consultaExame(ferimento)
    if exame not in listaExames:
      listaExames.append(exame)

  return listaExames

Função principal da aplicação, ira retornar o relatório da ocorrencia em formato Json com base na gravação do atendimento.

In [108]:
def gerarRelatorioDeOcorrencia():
  ocorrenciaJson = gerarOcorrencia()
  listaExames = geraListaExames(ocorrenciaJson["Ferimentos"])
  prompt = f"Complete a chave 'Exames' do seguinte JSON {ocorrenciaJson}, utilizando a lista  de exames {listaExames}."
  return chat.send_message(prompt).text

Executar programa!

In [109]:
print(gerarRelatorioDeOcorrencia())

```json
{
  "ResumoSituação": "Paciente do sexo masculino, aproximadamente 40 anos, encontrado consciente, porém confuso e desorientado após colisão veicular em alta velocidade.",
  "Ferimentos": [
    "Dificuldade respiratória",
    "Sinais de dor no peito",
    "Ferida profunda na testa com sangramento ativo",
    "Fratura exposta no braço direito"
  ],
  "TratamentosRealizados": [
    "Pressão na ferida da testa para controlar o sangramento",
    "Imobilização do braço fraturado",
    "Administração de oxigênio para auxiliar na respiração"
  ],
  "Exames": [
    "Avaliação clínica, biópsia",
    "Ultrassonografia (FAST), TC do abdômen",
    "Raio-X"
  ]
}
```

